In [3]:
import requests

def get_all_artifacts():
    artifacts = []
    page = 1
    while True:
        url = f"https://search.maven.org/solrsearch/select?q=g%3A%22*%22&rows=100&start={100 * (page - 1)}&wt=json"
        response = requests.get(url)
        data = response.json()

        for doc in data["response"]["docs"]:
            group_id = doc["g"]
            artifact_id = doc["a"]
            version = doc["latestVersion"]
            artifacts.append((group_id, artifact_id, version))

        # Check if there are more pages
        if (page * 100) >= data["response"]["numFound"]:
            break
        
        page += 1

    return artifacts

if __name__ == "__main__":
    all_artifacts = get_all_artifacts()
    for group_id, artifact_id, version in all_artifacts:
        print(f"GroupId: {group_id}, ArtifactId: {artifact_id}, Version: {version}")


In [4]:
import csv
import requests

def get_all_artifacts():
    artifacts = []
    page = 1
    while True:
        url = f"https://search.maven.org/solrsearch/select?q=g%3A%22*%22&rows=100&start={100 * (page - 1)}&wt=json"
        response = requests.get(url)
        data = response.json()

        for doc in data["response"]["docs"]:
            group_id = doc["g"]
            artifact_id = doc["a"]
            version = doc["latestVersion"]
            artifacts.append((group_id, artifact_id, version))

        # Check if there are more pages
        if (page * 100) >= data["response"]["numFound"]:
            break
        
        page += 1

    return artifacts

if __name__ == "__main__":
    all_artifacts = get_all_artifacts()
    with open("artifacts.csv", "w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["GroupId", "ArtifactId", "Version"])
        writer.writerows(all_artifacts)
